In [19]:
import pandas as pd

file1 = ("E://files_pandas//varios//ESTABLECIMIENTO_SALUD_202409.xlsx")
sheet = 'CLUES_202410'
clues_base = pd.read_excel(file1, sheet_name=sheet)
clues_base_imb = clues_base[
    (clues_base['CLAVE DE LA ENTIDAD'] == 13) & 
    (clues_base['ESTATUS DE OPERACION'] == 'EN OPERACION') & 
    (clues_base['CLAVE DE LA INSTITUCION'] == 'IMB')]

clues_base_coord = clues_base_imb[['CLUES','NOMBRE DE LA UNIDAD','LATITUD','LONGITUD']]

clues_ssa = clues_base[
    (clues_base['CLAVE DE LA ENTIDAD']== 13) &
    (clues_base['CLAVE DE LA INSTITUCION'] == 'SSA') & 
    (clues_base['COMENTARIOS DE LA VALIDACION'] == 'POR CAMBIO DE INSTITUCION, SE TRANSFIERE A SERVICIOS DE SALUD IMSS BIENESTAR')
]

clues_ssa_coord = clues_ssa[['CLUES','NOMBRE DE LA UNIDAD','LATITUD','LONGITUD']]




In [ ]:

clues_merge = pd.merge(
    clues_base_coord,
    clues_ssa_coord[['CLUES','NOMBRE DE LA UNIDAD']],
    on='NOMBRE DE LA UNIDAD',
    how='left',
    suffixes=('','_SSA')
)

columnas = ['CLUES', 'CLUES_SSA', 'NOMBRE DE LA UNIDAD','LATITUD', 'LONGITUD']
final_df = clues_merge[columnas]



In [30]:
unid_den = pd.read_excel('E://files_pandas//dengue//Dengue_Hgo.xlsx')

unid_den = unid_den.rename(columns={'CVE_UNI_MED_NOTIF':'CLUES_SSA'})

coord_merge = pd.merge(
    unid_den,
    final_df[['CLUES_SSA','LATITUD', 'LONGITUD']],
    on = 'CLUES_SSA',
    how= 'left',
    suffixes=('','_LOC')
    
)

df_unicos = coord_merge.drop_duplicates(subset=['LATITUD', 'LONGITUD'])
df_unicos = df_unicos.dropna(subset=['LATITUD', 'LONGITUD'])

In [ ]:

import folium
from folium.plugins import MarkerCluster
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from folium import LayerControl
import geopandas as gpd

try:
    gdf = gpd.read_file("E://files_pandas//sisver//muni_2018gw//muni_2018gw.shp", encoding='utf-8')
except UnicodeDecodeError:
    gdf = gpd.read_file("E://files_pandas//sisver//muni_2018gw//muni_2018gw.shp", encoding='latin-1')
#SELECCIONAR SOLO HIDALGO 
hgo = gdf[gdf['NOM_ENT'] == 'Hidalgo']


mapa = folium.Map(location=[df_unicos['LATITUD'].mean(), df_unicos['LONGITUD'].mean()], zoom_start=10)
heat_data = [[row['LATITUD'], row['LONGITUD'], row['VALOR']] for index, row in df_unicos.iterrows()]

# Añadir el HeatMap al mapa 
HeatMap(heat_data).add_to(mapa) # Guardar el mapa en un archivo HTML 
mapa.save('E://files_pandas//dengue//mapa_calor.html')


